# Full text search - Natalia Organek

In [1]:
import requests
import json
import os
import re

base_url = 'http://localhost:9200/'
headers = {'Content-Type': 'application/json'}

## Task 3
### Define an ES analyzer for Polish texts containing:
- standard tokenizer
- synonym filter with the following definitions:
 - kpk - kodeks postępowania karnego
 - kpc - kodeks postępowania cywilnego
 - kk - kodeks karny
 - kc - kodeks cywilny
- Morfologik-based lemmatizer
- lowercase filter

In [2]:
analyzer = {
    "analyzer": {
        "bill_analyzer": {
            "type": "custom", 
            "tokenizer": "standard",
            "filter": [
                "lowercase",     # lowercase as in task
                "codex_synonyms", # defined synonyms filter
                "morfologik_stem" # morfologik-based as in task
                ]
            }
        },
        "filter": {
            "codex_synonyms": {
                "type": "synonym_graph",
                "synonyms": [
                    "kpk, kodeks postępowania karnego",
                    "kpc, kodeks postępowania cywilnego",
                    "kk, kodeks karny",
                    "kc, kodeks cywilny"
                ]
            }
        }
}

## Task 4
### Define an ES index for storing the contents of the legislative acts.

In [3]:
index = {
    "settings": {
        "index": {
            "analysis": analyzer
        }
    },
    
    "mappings": { 
        "properties":{  
            "content": {
                "type": "text",
                "analyzer": "bill_analyzer"
            }
        }
    }
}

In [5]:
requests.delete(base_url + 'act-index')

<Response [200]>

In [6]:
r = requests.put(base_url + 'act-index', data=json.dumps(index), headers=headers)

In [7]:
r.text

'{"acknowledged":true,"shards_acknowledged":true,"index":"act-index"}'

Example using my analyzer:

In [8]:
body = {'analyzer': 'bill_analyzer', 'text': 'W KK w ust.5 art.13 stanowi się'}

In [9]:
r = requests.post(base_url + 'act-index/_analyze', data=json.dumps(body), headers=headers)

In [10]:
r.json()

{'tokens': [{'token': 'w',
   'start_offset': 0,
   'end_offset': 1,
   'type': '<ALPHANUM>',
   'position': 0},
  {'token': 'wiek',
   'start_offset': 0,
   'end_offset': 1,
   'type': '<ALPHANUM>',
   'position': 0},
  {'token': 'kodeks',
   'start_offset': 2,
   'end_offset': 4,
   'type': 'SYNONYM',
   'position': 1},
  {'token': 'kodeks karny',
   'start_offset': 2,
   'end_offset': 4,
   'type': '<ALPHANUM>',
   'position': 1,
   'positionLength': 2},
  {'token': 'karny',
   'start_offset': 2,
   'end_offset': 4,
   'type': 'SYNONYM',
   'position': 2},
  {'token': 'w',
   'start_offset': 5,
   'end_offset': 6,
   'type': '<ALPHANUM>',
   'position': 3},
  {'token': 'wiek',
   'start_offset': 5,
   'end_offset': 6,
   'type': '<ALPHANUM>',
   'position': 3},
  {'token': 'usta',
   'start_offset': 7,
   'end_offset': 10,
   'type': '<ALPHANUM>',
   'position': 4},
  {'token': '5',
   'start_offset': 11,
   'end_offset': 12,
   'type': '<NUM>',
   'position': 5},
  {'token': 'artyk

Text was lowercase, "KK" is kept as synonym of "kodeks karny", then synonyms were found.

## Task 5
### Load the data to the ES index.

In [13]:
file_regex = r'(\d+_\d+)\.txt'
directory = '../lab1/ustawy'
filenames = os.listdir(directory)


for filename in filenames:
    with open(os.path.join(directory, filename)) as f:
        bill = f.read()
        file_id = re.match(file_regex, filename).group(1)
        
        r = requests.put(base_url + 'act-index/_doc/' + file_id, json={'content': bill})
        if not 200 <= r.status_code < 300:
            print(file_id)
            print(r.text)
            print(r.status_code)
            break

In [12]:
requests.get(base_url + 'act-index/_doc/1993_599').json()

{'_index': 'act-index',
 '_type': '_doc',
 '_id': '1993_599',
 '_version': 1,
 '_seq_no': 103,
 '_primary_term': 1,
 'found': True,
 '_source': {'content': '\n\n\n\nDz.U. z 1993 r. Nr 129, poz. 599 \n                                USTAWA\n                        z dnia 9 grudnia 1993 r.\n                                    \n             o zmianie ustawy o podatku od towarów i usług\n                        oraz o podatku akcyzowym\n                                    \n                                    \n                                Art. 1.\nW ustawie z dnia 8 stycznia 1993 r. o podatku od towarów i usług oraz o podatku\nakcyzowym (Dz.U. Nr 11, poz. 50 i Nr 28, poz. 127) wprowadza się następujące\nzmiany:\n 1) w art. 3:\n    a) dotychczasowa treść otrzymuje oznaczenie ust. 1,\n    b) dodaje się ust. 2 w brzmieniu:\n          "2. Rada Ministrów może określać, w drodze rozporządzenia, \n             towary i usługi, z wyjątkiem wyrobów akcyzowych i wyrobów \n             z metali 

Bills were uploaded to index with accurate ids which are file names

## Task 6
### Determine the number of legislative acts containing the word ustawa (in any form).

In [14]:
query = {
    'query': {
        'match': {
            'content': 'ustawa'
        }
    }
}

In [15]:
r = requests.get(base_url + 'act-index/_search', json=query)

I take hits.total.value to get all acts in which 'ustawa' was found.

In [16]:
r.json()['hits']['total']['value']

1178

There are 1179 documents so one of them has not word 'ustawa'.

## Task 7
### Determine the number of occurrences of the word ustawa (in any form).

I use Termvectors API to get it. I create artificial document with only word ustawa to count total term frequency. [link](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-termvectors.html#docs-termvectors-artificial-doc)


In [48]:
query = {
  "doc" : {
    "content": "ustawa",
  },
  "term_statistics": True
}

In [49]:
r = requests.get(base_url + 'act-index/_termvectors', json=query)

In [50]:
r.json()

{'_index': 'act-index',
 '_type': '_doc',
 '_version': 0,
 'found': True,
 'took': 1,
 'term_vectors': {'content': {'field_statistics': {'sum_doc_freq': 705468,
    'doc_count': 1179,
    'sum_ttf': 5173547},
   'terms': {'ustawa': {'doc_freq': 1178,
     'ttf': 24934,
     'term_freq': 1,
     'tokens': [{'position': 0, 'start_offset': 0, 'end_offset': 6}]}}}}}

In [51]:
r.json()['term_vectors']['content']['terms']['ustawa']['ttf']

24934

It is 24934 times 'ustawa' in any form in bills.

If I choose 'ustaw'. it would be the same amount of words, because morfologic it would search it in any form. 

## Task 9
### Determine the number of legislative acts containing the words kodeks postępowania cywilnego in the specified order, but in any inflection form.

Match phrase to get them in specified order

In [95]:
query = {
    'query': {
        'match_phrase': {
            'content': 'kodeksu postępowania cywilnego'
        }
    },
    'highlight': {
        'fields': {
            'content': {
                'fragment_size':1,
                'type': 'plain'
            }
        }
    },
    'size':10
    
}

In [96]:
r = requests.get(base_url + 'act-index/_search', json=query)

In [97]:
r.json()

{'took': 126,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 99, 'relation': 'eq'},
  'max_score': 7.672101,
  'hits': [{'_index': 'act-index',
    '_type': '_doc',
    '_id': '2003_1611',
    '_score': 7.672101,
    '_source': {'content': '\n\n\nTekst ustawy przyjęty przez Senat bez poprawek\n\xa0\n\xa0\nustawa\nz dnia 22 lipca 2016 r.\no\nzmianie ustawy – Kodeks postępowania cywilnego, ustawy – Prawo o notariacie\noraz ustawy o zmianie ustawy – Kodeks cywilny, ustawy – Kodeks postępowania\ncywilnego oraz niektórych innych ustaw[1])\nArt.\xa01.\xa0W\nustawie z dnia 17 listopada 1964 r. – Kodeks postępowania cywilnego\n(Dz.\xa0U.\xa0z 2014 r. poz. 101, z późn. zm.[2])) w części czwartej w księdze drugiej w tytule VIIa\npo art. 11426 dodaje się art. 11427 w brzmieniu:\n„Art. 11427.\nPostanowienie o wydaniu, sprostowaniu, zmianie lub uchyleniu europejskiego\npoświadczenia spadkowego albo o zawieszeniu jego skutków o

In [90]:
r.json()['hits']['total']['value']

99

It's 99 occurences, but after seeing highlights, I see thet there could be also 'Kodeks' or 'postępowania cywilnego' phrases taken under consideration (and added to total). It is in correct order though.

## Task 10
### Determine the number of legislative acts containing the words wchodzi w życie (in any form) allowing for up to 2 additional words in the searched phrase.

'slop' argument allows additional words between phrase's words

In [102]:
query = {
    'query': {
        'match_phrase': {
            'content': {
                'query': 'wchodzi w życie',
                'slop': 2
            }
        }
    },
    'highlight': {
        'fields': {
            'content': {
                'fragment_size':1,
                'type': 'plain'
            }
        }
    },
    'size': 40
    
}

In [103]:
r = requests.get(base_url + 'act-index/_search', json=query)

In [104]:
r.json()

{'took': 177,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1174, 'relation': 'eq'},
  'max_score': 0.021548988,
  'hits': [{'_index': 'act-index',
    '_type': '_doc',
    '_id': '1997_940',
    '_score': 0.021548988,
    '_source': {'content': '\n\n\n\nDz.U. z 1998 r. Nr 99, poz. 631\n \n                                        \n                                        \n                                        \n                                     USTAWA\n                             z dnia 24 lipca 1998 r.\n                                        \n     o wejściu w życie ustawy o samorządzie powiatowym, ustawy o samorządzie\n         województwa oraz ustawy o administracji rządowej w województwie\n                                        \n                                     Art. 1.\n1. Ustawa z dnia 5 czerwca 1998 r. o samorządzie powiatowym (Dz.U. Nr 91, poz.\n  578), zwana dalej "ustawą o samorządzie powiat

Looking at 'highlights' - there are now e.g. counted:
'wchodzi w życie z'
'wchodzi w życie po'
'w życie nie'
'w życie w'

In [105]:
r.json()['hits']['total']['value']

1174

## Task 11
### Determine the 10 documents that are the most relevant for the phrase konstytucja.

In [134]:
query = {
    'query': {
        'match': {
            'content': 'konstytucja'
        }
    },
    'sort': ['_score'],
    'size': 10
}

I'm assuring there would be 10 bills, sorted by score (so relevance of word konstytucja)

In [135]:
r = requests.get(base_url + 'act-index/_search', json=query)

In [136]:
r.json()

{'took': 4,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 45, 'relation': 'eq'},
  'max_score': 6.869376,
  'hits': [{'_index': 'act-index',
    '_type': '_doc',
    '_id': '1997_629',
    '_score': 6.869376,
    '_source': {'content': '\n\n\n\nDz.U. z 1994 r. Nr 61, poz. 251\n                         USTAWA  KONSTYTUCYJNA\n                       z dnia 22 kwietnia 1994 r.\n                                    \n         o zmianie ustawy konstytucyjnej o trybie przygotowania\n           i uchwalenia Konstytucji Rzeczypospolitej Polskiej\n                                    \n                                    \n                                Art. 1.\nW ustawie  konstytucyjnej z  dnia 23 kwietnia 1992 r. o trybie przygotowania i \nuchwalenia Konstytucji Rzeczypospolitej Polskiej (Dz.U. Nr 67, poz. 336) \nwprowadza się następujące zmiany:\n \n 1) po art. 2 dodaje się art. 2a, art. 2b i art. 2c w brzmieniu:\n   "Ar

In [138]:
[(bill['_id'], bill['_score']) for bill in r.json()['hits']['hits']]

[('1997_629', 6.869376),
 ('2000_443', 6.6642833),
 ('1997_604', 6.633483),
 ('1996_350', 6.628302),
 ('1997_642', 6.2530584),
 ('2001_23', 6.0589767),
 ('1996_199', 5.9289904),
 ('1999_688', 5.8507533),
 ('1997_681', 5.467437),
 ('2001_1082', 5.467437)]

## Task 12

In [140]:
query = {
    'query': {
        'match': {
            'content': 'konstytucja'
        }
    },
    'highlight': {
        'fields': {
            'content': {
                'number_of_fragments': 3
            }
        }
    },
    'sort': ['_score'],
    'size': 10
}

In [141]:
r = requests.get(base_url + 'act-index/_search', json=query)

In [152]:
occs = [(bill['_id'], bill['highlight']['content']) for bill in r.json()['hits']['hits']]

In [153]:
for occ in occs:
    print(occ[0])
    for occ_f in occ[1]:
        print(occ_f)
    print('----------------')

1997_629
o zmianie ustawy konstytucyjnej o trybie przygotowania
           i uchwalenia <em>Konstytucji</em> Rzeczypospolitej
W ustawie  konstytucyjnej z  dnia 23 kwietnia 1992 r. o trybie przygotowania i 
uchwalenia <em>Konstytucji</em>
Do zgłoszenia projektu <em>Konstytucji</em> załącza się wykaz 
                obywateli popierających zgłoszenie
----------------
2000_443
umowy międzynarodowej i nie wypełnia przesłanek określonych w art. 89
     ust. 1 lub art. 90 <em>Konstytucji</em>
międzynarodowej lub załącznika nie
     wypełnia przesłanek określonych w art. 89 ust. 1 lub art. 90 <em>Konstytucji</em>
co do zasadności wyboru
  trybu ratyfikacji umowy międzynarodowej, o którym mowa w art. 89 ust. 2
  <em>Konstytucji</em>
----------------
1997_604
Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności celów partii 
   politycznej z <em>Konstytucją</em>
Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności z <em>Konstytucją</em>
   celów lub działalności
Ciężar udowodni

Many of those acts are from about 1997 year so preparing and settling Polish Constitution from April 1997